In [2]:
using Gurobi;
using MaxEntChemostat2018;
using DataFrames;
using JuMP;

INFO: Precompiling module CPLEX.
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stderr,Timer})() at ./event.jl:436
while loading /Users/Pereiro/.julia/v0.6/COBRA/src/COBRA.jl, in expression starting on line 27
ERROR: LoadError: CPLEX not properly installed. Please run Pkg.build("CPLEX")
Stacktrace:
 [1] error(::String) at ./error.jl:21
 [2] include_from_node1(::String) at ./loading.jl:576
 [3] include(::String) at ./sysimg.jl:14
 [4] anonymous at ./<missing>:2
while loading /Users/Pereiro/.julia/v0.6

In [3]:
S, mets, rxns = MaxEntChemostat2018.FBA.load_ecoli();

In [4]:
function fba(S, mets, rxns; 
        objtindex::Int = 13, C = 0.0, 
        add_bound_constraints::Bool = true)::FBAResult
    
    #Checking
    @assert all(size(S) .== (size(mets,1), size(rxns,1)))
    @assert allunique(rxns[:id])
    @assert allunique(mets[:id])
    C > 0 && @assert !all(names(rxns) .!= :an) && !all(names(rxns) .!= :ap)
    add_bound_constraints && @assert !all(names(rxns) .!= :lb) && !all(names(rxns) .!= :ub)
    
    # Sizes
    rscount = size(rxns,1);
    mscount = size(mets,1);
    
    #model
    model = Model();
    setsolver(model, Gurobi.GurobiSolver());
    
    #variables
    pfluxes = Vector{JuMP.Variable}();
    nfluxes = Vector{JuMP.Variable}();
    for ri in 1:rscount
        
        #positive fluxes
        var = @variable(model, basename = "p_$(rxns[:id][ri])");
        push!(pfluxes,var);
        
        #negative fluxes
        var = @variable(model, basename = "n_$(rxns[:id][ri])");
        push!(nfluxes, var);
        
    end
    #Objective flux
    obj = pfluxes[objtindex];
    
    #Bound Constraints
    if add_bound_constraints
        for ri in 1:rscount
            @constraint(model, pfluxes[ri] >= 0);
            @constraint(model, pfluxes[ri] <= rxns[:ub][ri]);
            @constraint(model, nfluxes[ri] >= 0);
            @constraint(model, nfluxes[ri] <= -rxns[:lb][ri]);
        end
    end
    
    #Balance Constraints
    for mi in 1:mscount
        @constraint(model, S[mi,:]' * (pfluxes - nfluxes) == 0);
    end
    
    #Cost constraints
    if C > 0
        @constraint(model, sum(rxns[:ap]' * pfluxes + rxns[:an]' * nfluxes) <= C) 
    end
    
    #Objective
    @objective(model, Max, obj);
    
    #Solving
    solve(model);
    
    return FBAResult(model, pfluxes, nfluxes);
    
end

struct FBAResult
    model::JuMP.Model
    pfluxes::Vector{JuMP.Variable}
    nfluxes::Vector{JuMP.Variable}
end

In [5]:
result = fba(S,mets, rxns; C = 1);

Academic license - for non-commercial use only
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 453 rows, 190 columns and 1290 nonzeros
Coefficient statistics:
  Matrix range     [1e-07, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 412 rows and 99 columns
Presolve time: 0.00s
Presolved: 41 rows, 91 columns, 427 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.2381070e+01   6.424725e+02   0.000000e+00      0s
      53    1.9725678e-01   0.000000e+00   0.000000e+00      0s

Solved in 53 iterations and 0.00 seconds
Optimal objective  1.972567824e-01


## Tools

In [6]:
"""
Returns a DataFrame with the result
"""
function Base.convert(::Type{DataFrames.DataFrame}, vars::Array{JuMP.Variable,1})::DataFrame
    df = DataFrames.DataFrame(); 
    df[:id] = string.(vars);
    df[:v] = Float64.(getvalue.(vars))
    return df;
end
function Base.convert(::Type{DataFrames.DataFrame}, fbares::FBAResult)::DataFrame
    df = DataFrames.DataFrame();
    df[:id] = [string.(fbares.pfluxes) ; string.(fbares.nfluxes)]
    df[:v] = [Float64.(getvalue.(fbares.pfluxes)) ; Float64.(getvalue.(fbares.nfluxes))];
    return df;
end
function get_non_zeros(S, mets, rxns, fbares::FBAResult)
    nz_index = Vector{Int}();
    for ri in 1:size(rxns,1)
        if (getvalue(fbares.pfluxes[ri]) - getvalue(fbares.nfluxes[ri])) != 0.0 
            push!(nz_index, ri);
        end
    end
    
    nz_fba_res = FBAResult(fbares.model, fbares.pfluxes[nz_index], fbares.nfluxes[nz_index]);

    nz_rxns = rxns[nz_index,:];
    nz_S = S[:,nz_index];

    nz_index = Vector{Int}();
    for mi in 1:size(mets,1)
        if length(nonzeros(nz_S[mi,:])) != 0
           push!(nz_index, mi);
        end
    end
    nz_mets = mets[nz_index,:];
    nz_S = nz_S[nz_index,:];
    
    
    return nz_S, nz_mets, nz_rxns, nz_fba_res
end

convert (generic function with 833 methods)

,x1,x2,x3,x4,x5
1,0.8059,0.144683,0.754424,0.214457,0.483083
2,0.294356,0.65448,0.697482,0.0817572,0.241229
3,0.895674,0.0109339,0.0991546,0.446525,0.485647
4,0.755767,0.181588,0.187046,0.201375,0.406749
5,0.243551,0.355852,0.185825,0.065589,0.359087


In [8]:
convert

search: convert ConjVector code_native @code_native



Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading /Users/Pereiro/.julia/v0.6/IJulia/src/kernel.jl, in expression starting on line 188


```
convert(T, x)
```

Convert `x` to a value of type `T`.

If `T` is an [`Integer`](@ref) type, an [`InexactError`](@ref) will be raised if `x` is not representable by `T`, for example if `x` is not integer-valued, or is outside the range supported by `T`.

# Examples

```jldoctest
julia> convert(Int, 3.0)
3

julia> convert(Int, 3.5)
ERROR: InexactError()
Stacktrace:
 [1] convert(::Type{Int64}, ::Float64) at ./float.jl:679
```

If `T` is a [`AbstractFloat`](@ref) or [`Rational`](@ref) type, then it will return the closest value to `x` representable by `T`.

```jldoctest
julia> x = 1/3
0.3333333333333333

julia> convert(Float32, x)
0.33333334f0

julia> convert(Rational{Int32}, x)
1//3

julia> convert(Rational{Int64}, x)
6004799503160661//18014398509481984
```

If `T` is a collection type and `x` a collection, the result of `convert(T, x)` may alias `x`.

```jldoctest
julia> x = Int[1,2,3];

julia> y = convert(Vector{Int}, x);

julia> y === x
true
```

Similarly, if `T` is a composite type and `x` a related instance, the result of `convert(T, x)` may alias part or all of `x`.

```jldoctest
julia> x = speye(5);

julia> typeof(x)
SparseMatrixCSC{Float64,Int64}

julia> y = convert(SparseMatrixCSC{Float64,Int64}, x);

julia> z = convert(SparseMatrixCSC{Float32,Int64}, y);

julia> y === x
true

julia> z === x
false

julia> z.colptr === x.colptr
true
```

`convert(StringArray{U}, A::StringArray{T})`

convert `A` to StringArray of another element type (`U`) without copying the underlying data.

Returns a DataFrame with the result


In [9]:
function get_non_zeros(S, mets, rxns, fbares::FBAResult)
    nz_index = Vector{Int}();
    for ri in 1:size(rxns,1)
        if (getvalue(fbares.pfluxes[ri]) - getvalue(fbares.nfluxes[ri])) != 0.0 
            push!(nz_index, ri);
        end
    end
    
    nz_fba_res = FBAResult(fbares.model, fbares.pfluxes[nz_index], fbares.nfluxes[nz_index]);

    nz_rxns = rxns[nz_index,:];
    nz_S = S[:,nz_index];

    nz_index = Vector{Int}();
    for mi in 1:size(mets,1)
        if length(nonzeros(nz_S[mi,:])) != 0
           push!(nz_index, mi);
        end
    end
    nz_mets = mets[nz_index,:];
    nz_S = nz_S[nz_index,:];
    
    
    return nz_S, nz_mets, nz_rxns, nz_fba_res
end

get_non_zeros (generic function with 1 method)

In [10]:
nz_S, nz_mets, nz_rxns, nz_fba_res = get_non_zeros(S, mets, rxns, fba(S, mets, rxns; C = 0.005));

Academic license - for non-commercial use only
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 453 rows, 190 columns and 1290 nonzeros
Coefficient statistics:
  Matrix range     [1e-07, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+03]
Presolve removed 412 rows and 99 columns
Presolve time: 0.00s
Presolved: 41 rows, 91 columns, 430 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1290035e-01   1.721622e+01   0.000000e+00      0s
      53    9.8628391e-04   0.000000e+00   0.000000e+00      0s

Solved in 53 iterations and 0.00 seconds
Optimal objective  9.862839119e-04


In [11]:
get_result_df(nz_fba_res.nfluxes)

LoadError: [91mUndefVarError: get_result_df not defined[39m

In [12]:
a = [1,2,3];
b = [4,5,6];
[println("$i , $j") for i in a for j in b]

1 , 4
1 , 5
1 , 6
2 , 4
2 , 5
2 , 6
3 , 4
3 , 5
3 , 6


9-element Array{Void,1}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing